In [1]:
#from scraper import fetch_website_contents
from openai import OpenAI
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama=OpenAI(base_url=OLLAMA_BASE_URL,api_key="ollama")

In [2]:
system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

In [3]:
#fetch website content
def messages_for(website):
    return[{"role":"system","content":system_prompt},
           {"role":"user","content":user_prompt_prefix+website}]
    
    

In [4]:
#readwebsite
from bs4 import BeautifulSoup
import requests

# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]

In [5]:
ed="https://myknowtech.com/"
messages_for(ed)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\n\nhttps://myknowtech.com/'}]

In [8]:
def summarize(url):
    website=fetch_website_contents(url)
    response=ollama.chat.completions.create(model="llama3.2:1b",messages=messages_for(website))
    return response.choices[0].message.content

In [ ]:
summarize("https://myknowtech.com/")

In [40]:
summarize("https://www.udemy.com/")

'Ah, the classic "Enable JavaScript and cookies" roadblock—because nothing says "welcome" like a digital bouncer demanding your browser obey its every whim before you can even see what\'s inside. Spoiler alert: I can\'t summarize what I can\'t see, so enjoy toggling those settings!'